In [2]:
import os
import shutil
from PIL import Image
from matplotlib import pyplot
import numpy as np
import cv2
from keras_facenet import FaceNet
import dlib
import random
# "D:\IntegrationXpert\male, female & kids\dataset\training\male"

In [ ]:
# This code deletes the augmented images and saves only one unique image

def delete_duplicate_starting_files(directory):
    starting_name_dict = {}
    for filename in os.listdir(directory):
        starting_name = filename.split('_')[0]
        
        if starting_name in starting_name_dict:
            starting_name_dict[starting_name].append(filename)
        else:

            starting_name_dict[starting_name] = [filename]

    for starting_name, files in starting_name_dict.items():
        random.shuffle(files)
        files_to_delete = files[1:]
        for file_to_delete in files_to_delete:
            file_path = os.path.join(directory, file_to_delete)
            try:
                os.remove(file_path)
                print(f"Deleted: {file_to_delete}")
            except OSError as e:
                print(f"Error deleting file {file_to_delete}: {e}")

# Example usage
directory = r"D:\IntegrationXpert\male, female & kids\dataset\temp"
delete_duplicate_starting_files(directory)


In [3]:
# This code detects faces and saves it into another directory

def generate_unique_filename(output_dir, base_name, extension, i):
    counter = 0
    while True:
        if counter == 0:
            new_name = f"{base_name}{extension}"
        else:
            new_name = f"{base_name}{counter}{extension}"
        new_path = os.path.join(output_dir, new_name)
        if not os.path.exists(new_path):
            return new_path
        counter += 1

def detect_and_crop_faces_in_directory(source_directory, output_dir, padding=20, top_padding=40, scale_factor=1.0):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    detector = dlib.get_frontal_face_detector()

    for filename in os.listdir(source_directory):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(source_directory, filename)


            image = cv2.imread(image_path)
            if image is None:
                print(f"Error loading image: {image_path}")
                continue
            name_without_extension = os.path.splitext(filename)[0]

            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            faces = detector(gray_image)

            for i, face in enumerate(faces):
                x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
                x1, y1 = max(0, int(x1) - padding), max(0, int(y1) - top_padding)
                x2, y2 = min(int(x2) + padding, image.shape[1]), min(int(y2) + padding, image.shape[0])
                cropped_face = image[y1:y2, x1:x2]

                face_filename = generate_unique_filename(output_dir, name_without_extension, ".jpg", i)
                cv2.imwrite(face_filename, cropped_face)

            print(f"{len(faces)} face(s) found and saved from {filename} in {output_dir}")
        else:
            print(f"Skipping non-image file: {filename}")
source_directory = r"D:\IntegrationXpert\male, female & kids\vgg 16\test_images"
destination_directory = r"D:\IntegrationXpert\male, female & kids\vgg 16\temp"
detect_and_crop_faces_in_directory(source_directory, destination_directory)


1 face(s) found and saved from image1.jpg in D:\IntegrationXpert\male, female & kids\vgg 16\temp
1 face(s) found and saved from image2.jpeg in D:\IntegrationXpert\male, female & kids\vgg 16\temp
0 face(s) found and saved from image3.jpeg in D:\IntegrationXpert\male, female & kids\vgg 16\temp
0 face(s) found and saved from image4.jpeg in D:\IntegrationXpert\male, female & kids\vgg 16\temp
0 face(s) found and saved from image5.jpeg in D:\IntegrationXpert\male, female & kids\vgg 16\temp
0 face(s) found and saved from image6.jpeg in D:\IntegrationXpert\male, female & kids\vgg 16\temp


In [158]:
# This code saves the face cropped images into our dataset by making sure every image has a unique name

def rename_and_save_files(source_directory, destination_directory):
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    # Iterate through all files in the source directory
    for filename in os.listdir(source_directory):
        if filename.lower().endswith(".jpg"):
            parts = filename.split('_')
            if len(parts) > 1:
                new_filename = parts[0] + '.jpg'
                old_file = os.path.join(source_directory, filename)
                new_file = os.path.join(destination_directory, new_filename)

                while os.path.exists(new_file):
                    random_number = random.randint(1000, 9999)
                    new_filename = parts[0] + f'_{random_number}.jpg'
                    new_file = os.path.join(destination_directory, new_filename)
                shutil.copy2(old_file, new_file)
                print(f'Renamed: {filename} to {new_filename} and saved to {destination_directory}')

# Example usage
source_directory = r"D:\IntegrationXpert\male, female & kids\dataset\temp2"
destination_directory = r"D:\IntegrationXpert\male, female & kids\dataset\training\kid"
rename_and_save_files(source_directory, destination_directory)
#D:\IntegrationXpert\male, female & kids\dataset\validation\kid

In [151]:
# This code extracts images from the UTKdataset
# Edit the prefix as the age you want

def copy_files_with_prefix(source_directory, destination_directory, prefix='10_'):
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    for filename in os.listdir(source_directory):
        if filename.startswith(prefix):
            source_path = os.path.join(source_directory, filename)
            
            base_name = filename.split('.')[0]
            new_filename = f"{base_name}.jpg"
            destination_path = os.path.join(destination_directory, new_filename)
            
            shutil.copy2(source_path, destination_path)
            print(f"Copied {filename} to {destination_directory} as {new_filename}")
            
source_directory = r"C:\Users\User\Downloads\UTKFace"
destination_directory = r"D:\IntegrationXpert\male, female & kids\dataset\temp2"
copy_files_with_prefix(source_directory, destination_directory)


In [166]:
# This code splits the data

def move_random_images(source_directory, destination_directory, percentage=0.5):
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    all_files = [f for f in os.listdir(source_directory) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    num_files_to_move = int(len(all_files) * percentage)

    files_to_move = random.sample(all_files, num_files_to_move)

    for filename in files_to_move:
        source_path = os.path.join(source_directory, filename)
        destination_path = os.path.join(destination_directory, filename)
        shutil.move(source_path, destination_path)
        print(f"Moved {filename} to {destination_directory}")

source_directory = r"D:\IntegrationXpert\male, female & kids\dataset\validation\kid"
destination_directory = r"D:\IntegrationXpert\male, female & kids\dataset\validation\k_t"
move_random_images(source_directory, destination_directory)


Moved child-1469.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved 1_9486.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved 10_2129.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved 7_7005.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved 41.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved 5_5231.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved 5_1473.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved 9_8184.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved 1.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved 18039.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved child-1187.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
Moved child-2102.jpg to D:\IntegrationXpert\male, female & kids\dataset\validation\k_t
M